"""
**Data Preprocessing for Movie Lens Dataset**

The Movie Lens dataset is a popular dataset used for recommendation systems and movie ratings analysis. This code aims to preprocess the dataset before further analysis or modeling.

**Preprocessing Steps:**

1. **Loading the dataset:** The code loads the Movie Lens dataset from a specific file path.

2. **Handling missing values:** The code handles any missing values in the dataset by applying appropriate techniques such as imputation or removal.

3. **Removing duplicates:** The code identifies and removes any duplicate entries in the dataset to ensure data integrity.

4. **Encoding categorical variables:** The code encodes categorical variables in the dataset into numerical representations, making them suitable for analysis or modeling.

5. **Scaling numerical variables:** The code scales numerical variables in the dataset to a common range, ensuring that they have a similar impact during analysis or modeling.

By performing these preprocessing steps, the Movie Lens dataset is prepared for further analysis or modeling tasks.

**Author:** [Your Name]

**Date:** [Current Date]

In [1]:
#imports
import pandas as pd

In [2]:
#Loading Dataset
ratings = pd.read_csv('Data/ratings.csv')
tags = pd.read_csv('Data/tags.csv')
movies = pd.read_csv('Data/movies.csv')
links = pd.read_csv('Data/links.csv')
genome_scores = pd.read_csv('Data/genome-scores.csv')
genome_tags = pd.read_csv('Data/genome-tags.csv')
